**Mounting the drive**

In [294]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Importing important libraries**

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import time

**Reading and removing incomplete data from dataset Housing.csv**

In [0]:
dataset = pd.read_csv('/content/drive/My Drive/housing.csv')
dataset = dataset.dropna()

**Printing the first ten records from dataset**

In [0]:
print("Here are the first ten rows of dataset:")
dataset.head(10)

**Plotting the features in a single graph**

In [0]:
plt.figure()
dataset.plot(subplots=True,figsize=(10,10))

**Processing the dataset**

In [0]:
Y = dataset["median_house_value"]
#From column longitude to median income will predict the value of Y
X = dataset.loc[:,'longitude':'median_income']

#Here I have split the training and testing dataset ratio is set to 70:30 and set the random state
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state = 2003)

#To use Pytorch library we convert dataset to numpy
x_train_np = x_train.to_numpy()
y_train_np = y_train.to_numpy()

x_test_np = x_test.to_numpy()
y_test_np = y_test.to_numpy()

**Importing libraries for creating model**

In [0]:
import torch
from torch.nn import Conv1d   #Importing 1D convolution layer
from torch.nn import Dropout  #Importing dropout layer to handling the overfitting issues in the code
from torch.nn import MaxPool1d  #Importing the max pooling layer
from torch.nn import Flatten  #Importing the flatten layer
from torch.nn import Linear   #Importing the linear layer
from torch.nn.functional import relu  #Importing the relu activation function
from torch.utils.data import DataLoader, TensorDataset

**Creating the neural network model**

In [0]:
class CnnRegressor(torch.nn.Module):
  def __init__(self, batch_size, inputs, outputs):
    super(CnnRegressor, self).__init__()
    self.batch_size = batch_size
    self.inputs = inputs
    self.outputs = outputs
    self.dropout = Dropout(0.8)
    self.input_layer = Conv1d(inputs, batch_size, 1)
    self.max_pooling_layer = MaxPool1d(1)
    self.conv_layer = Conv1d(batch_size, 256, 1)
    self.flatten_layer = Flatten()
    self.linear_layer = Linear(256, 64)
    self.output_layer = Linear(64, outputs)
  
  def feed(self, input):
    input = input.reshape((self.batch_size, self.inputs, 1))
    output = relu(self.input_layer(input))
    output = self.max_pooling_layer(output)
    output = relu(self.conv_layer(output))
    output = self.flatten_layer(output)
    output = self.linear_layer(output)
    output = self.dropout(output)
    output = self.output_layer(output)
    return output

**Importing Libraries and R2Score**

In [302]:
from torch.optim import SGD
from torch.optim import Adadelta
from torch.optim import Adam
from torch.optim import Adagrad
from torch.nn import L1Loss
!pip install pytorch-ignite
from ignite.contrib.metrics.regression.r2_score import R2Score

**Defining the model**

In [303]:
batch_size = 64
model = CnnRegressor(batch_size, X.shape[1], 1)
model.cuda()

CnnRegressor(
  (dropout): Dropout(p=0.8, inplace=False)
  (input_layer): Conv1d(8, 64, kernel_size=(1,), stride=(1,))
  (max_pooling_layer): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_layer): Conv1d(64, 256, kernel_size=(1,), stride=(1,))
  (flatten_layer): Flatten()
  (linear_layer): Linear(in_features=256, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

In [0]:
def model_loss(model, dataset, train = False, optimizer = None):
  performance = L1Loss()
  score_metric = R2Score()

  avg_loss = 0
  avg_score = 0
  count = 0
  for input, output in iter(dataset):
    
    #Getting the model's preidctions for training dataset
    predictions = model.feed(input)

    #Getting the model loss
    loss = performance(predictions, output)
    
    #Getting the model's R2 score
    score_metric.update([predictions, output])
    score = score_metric.compute()

    if(train):
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
    avg_loss += loss.item()
    avg_score += score
    count += 1
  return avg_loss / count, avg_score / count

**Training the model for dataset**

In [0]:
#Defining the number of epochs to be trained
epochs = 500

#Defining the optimizer 
#optimizer = SGD(model.parameters(), lr=1e-5)
#optimizer= Adadelta(model.parameters(), lr=0.1, rho=0.4, eps=1e-06, weight_decay=0)
#optimizer = Adam(model.parameters())
optimizer = Adagrad(model.parameters(), lr=0.3, eps=1e-10,weight_decay=0, initial_accumulator_value=0)
#Converting testing set into torch variable as floats
inputs = torch.from_numpy(x_train_np).cuda().float()
outputs = torch.from_numpy(y_train_np.reshape(y_train_np.shape[0], 1)).cuda().float()

#Creating DataLoader instance
tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)

for epoch in range(epochs):
  avg_loss, avg_r2_score = model_loss(model, loader, train=True, optimizer=optimizer)

  print("Epoch " + str(epoch + 1) + ":\n\tLoss = " + str(avg_loss) + "\n\tR^2 Score = " + str(avg_r2_score))

**Testing the model and Showing the final results with inference time for testing** 

In [0]:
inputs = torch.from_numpy(x_test_np).cuda().float()
outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0],1)).cuda().float()

#Inference time for testing
start_time = time.time()

tensor = TensorDataset(inputs,outputs)
loader = DataLoader(tensor,batch_size,shuffle=True,drop_last=True)

avg_loss, avg_r2_score = model_loss(model, loader)
print("The model's L1lossis:"+str(avg_loss))
print("The model's R^2scoreis:"+str(avg_r2_score))
print("--- %s seconds ---" % (time.time() - start_time))